# 04_model_training_evaluation.ipynb

# Importando as bibliotecas necessárias

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import joblib # Para salvar e carregar modelos

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Bibliotecas importadas com sucesso!")

Bibliotecas importadas com sucesso!


# --- 1. Carregamento do Dataset Processado da Fase ANALYZE ---

In [2]:
url = 'https://raw.githubusercontent.com/moises-rb/projeto_futebol_preditivo/refs/heads/main/03_analyze/data/processed/analyzed_data.csv'
try:
    df = pd.read_csv(url)
    print(f"\nDataset 'analyzed_data.csv' carregado com sucesso!")
    print("Primeiras 5 linhas do dataset:")
    print(df.head())
    print("\nInformações básicas do dataset:")
    df.info()
except FileNotFoundError:
    print(f"\nErro: O arquivo '{url}' não foi encontrado.")
    print("Por favor, certifique-se de que o arquivo está no caminho correto.")
    print("Você pode precisar executar o notebook '03_feature_engineering_correlation.ipynb' primeiro para gerar este arquivo.")


Dataset 'analyzed_data.csv' carregado com sucesso!
Primeiras 5 linhas do dataset:
         date home_team away_team  home_score  away_score tournament     city  \
0  1872-11-30  Scotland   England           0           0   Friendly  Glasgow   
1  1873-03-08   England  Scotland           4           2   Friendly   London   
2  1874-03-07  Scotland   England           2           1   Friendly  Glasgow   
3  1875-03-06   England  Scotland           2           2   Friendly   London   
4  1876-03-04  Scotland   England           3           0   Friendly  Glasgow   

    country  neutral    result  year  month  day_of_week  is_home_game  \
0  Scotland    False      Draw  1872     11            5             1   
1   England    False  Home Win  1873      3            5             1   
2  Scotland    False  Home Win  1874      3            5             1   
3   England    False      Draw  1875      3            5             1   
4  Scotland    False  Home Win  1876      3            5    

# --- 2. Preparação dos Dados para Modelagem ---

In [3]:
# Definindo as features (X) e a variável alvo (y)
# Remover 'date', 'home_score', 'away_score' e 'result_numeric' (já que 'result' é a categórica alvo)
# 'city', 'country', 'tournament' são categóricas e precisarão de One-Hot Encoding
features = ['home_team', 'away_team', 'tournament', 'city', 'country',
            'neutral', 'year', 'month', 'day_of_week', 'is_home_game',
            'goal_difference', 'total_goals']
target = 'result' # 'Home Win', 'Away Win', 'Draw'

X = df[features]
y = df[target]

In [4]:
# Identificando colunas numéricas e categóricas
numerical_cols = ['year', 'month', 'day_of_week', 'goal_difference', 'total_goals']
categorical_cols = ['home_team', 'away_team', 'tournament', 'city', 'country', 'neutral', 'is_home_game']

In [5]:
# Criando um pré-processador para aplicar transformações diferentes a colunas diferentes
# Standard Scaler para colunas numéricas
# OneHotEncoder para colunas categóricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ])

In [6]:
# Divisão do dataset em conjuntos de treino e teste
# Usaremos stratify=y para garantir que a proporção das classes da variável alvo seja mantida
# em ambos os conjuntos (treino e teste), o que é importante para problemas de classificação.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Tamanho do conjunto de treino: {X_train.shape[0]} amostras")
print(f"Tamanho do conjunto de teste: {X_test.shape[0]} amostras")
print("\nDistribuição das classes no conjunto de treino:")
print(y_train.value_counts(normalize=True))
print("\nDistribuição das classes no conjunto de teste:")
print(y_test.value_counts(normalize=True))

Tamanho do conjunto de treino: 38668 amostras
Tamanho do conjunto de teste: 9667 amostras

Distribuição das classes no conjunto de treino:
result
Home Win    0.490380
Away Win    0.282197
Draw        0.227423
Name: proportion, dtype: float64

Distribuição das classes no conjunto de teste:
result
Home Win    0.490431
Away Win    0.282197
Draw        0.227371
Name: proportion, dtype: float64


# --- 3. Seleção e Treinamento de Modelos de Machine Learning ---

In [7]:
# Modelo 1: Regressão Logística
# Um pipeline para pré-processamento e modelo
pipeline_lr = Pipeline(steps=[('preprocessor', preprocessor),
                              ('classifier', LogisticRegression(solver='liblinear', random_state=42, max_iter=1000))])

print("\nTreinando Regressão Logística...")
pipeline_lr.fit(X_train, y_train)
print("Regressão Logística treinada!")


Treinando Regressão Logística...
Regressão Logística treinada!


In [8]:
# Modelo 2: Random Forest Classifier
pipeline_rf = Pipeline(steps=[('preprocessor', preprocessor),
                              ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1))])

print("\nTreinando Random Forest...")
pipeline_rf.fit(X_train, y_train)
print("Random Forest treinada!")


Treinando Random Forest...
Random Forest treinada!


# --- 4. Avaliação do Desempenho do Modelo ---

In [12]:
models = {
    'Logistic Regression': pipeline_lr,
    'Random Forest': pipeline_rf
}

best_model_name = None
best_accuracy = 0
best_model = None

for name, model in models.items():
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"\n--- {name} ---")
    print(f"Acurácia: {accuracy:.4f}")
    print("\nRelatório de Classificação:")
    print(classification_report(y_test, y_pred))
    print("\nMatriz de Confusão:")
    print(confusion_matrix(y_test, y_pred))

    # Atualizar o melhor modelo
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model_name = name
        best_model = model

print(f"\n--- Melhor Modelo: {best_model_name} com Acurácia de {best_accuracy:.4f} ---")


--- Logistic Regression ---
Acurácia: 0.9993

Relatório de Classificação:
              precision    recall  f1-score   support

    Away Win       1.00      1.00      1.00      2728
        Draw       1.00      1.00      1.00      2198
    Home Win       1.00      1.00      1.00      4741

    accuracy                           1.00      9667
   macro avg       1.00      1.00      1.00      9667
weighted avg       1.00      1.00      1.00      9667


Matriz de Confusão:
[[2728    0    0]
 [   7 2191    0]
 [   0    0 4741]]

--- Random Forest ---
Acurácia: 0.9983

Relatório de Classificação:
              precision    recall  f1-score   support

    Away Win       1.00      1.00      1.00      2728
        Draw       1.00      0.99      1.00      2198
    Home Win       1.00      1.00      1.00      4741

    accuracy                           1.00      9667
   macro avg       1.00      1.00      1.00      9667
weighted avg       1.00      1.00      1.00      9667


Matriz de Confusã

# --- 5. Interpretação do Modelo (para Random Forest) ---

    # Obter os nomes das features após o OneHotEncoding
    # Isso é um pouco mais complexo, pois o OneHotEncoder cria muitas colunas.
    # Vamos focar nas importâncias das features numéricas e nas principais categóricas.

In [15]:
print(f"\nModelo selecionado para interpretação: {best_model_name}") # Nova linha para depuração

if best_model_name == 'Random Forest':
    print("Entrou no bloco de geração de gráficos para Random Forest.") # Nova linha para depuração
    print("\n--- Importância das Features (Random Forest) ---")
    # Acessar o Random Forest Classifier dentro do pipeline que foi identificado como o melhor modelo
    # best_model é o pipeline completo (pipeline_rf neste caso)
    rf_classifier = best_model.named_steps['classifier'] # Usando best_model para garantir que é o pipeline correto

    # Obter os nomes das features após o OneHotEncoding
    ohe_feature_names = best_model.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(categorical_cols)
    all_feature_names = numerical_cols + list(ohe_feature_names)

    # Criar um DataFrame para as importâncias
    feature_importances = pd.Series(rf_classifier.feature_importances_, index=all_feature_names)

    # Exibir as top N features mais importantes
    top_n = 15 # Ajuste conforme necessário
    print(feature_importances.nlargest(top_n))

    plt.figure(figsize=(12, 8))
    sns.barplot(x=feature_importances.nlargest(top_n).values, y=feature_importances.nlargest(top_n).index, palette='viridis')
    plt.title(f'Top {top_n} Features Mais Importantes (Random Forest)')
    plt.xlabel('Importância')
    plt.ylabel('Feature')
    plt.show()
else: # Adicionado para depuração
    print("Random Forest não foi o melhor modelo. Gráfico de importância de features não gerado.")


Modelo selecionado para interpretação: Logistic Regression
Random Forest não foi o melhor modelo. Gráfico de importância de features não gerado.


In [14]:
# --- 6. Salvamento do Melhor Modelo ---
output_model_dir = '../models/'
if not os.path.exists(output_model_dir):
    os.makedirs(output_model_dir)
    print(f"Diretório '{output_model_dir}' criado com sucesso para salvar modelos.")

model_filename = os.path.join(output_model_dir, f'{best_model_name.replace(" ", "_").lower()}_model.joblib')
joblib.dump(best_model, model_filename)
print(f"\nMelhor modelo ('{best_model_name}') salvo em: {model_filename}")

print("\nFase IMPROVE (Treinamento e Avaliação de Modelos) concluída neste notebook.")
print("Pronto para aprofundar no Controle (Fase CONTROL)!")


Melhor modelo ('Logistic Regression') salvo em: ../models/logistic_regression_model.joblib

Fase IMPROVE (Treinamento e Avaliação de Modelos) concluída neste notebook.
Pronto para aprofundar no Controle (Fase CONTROL)!
